# Tugas 6 : Query dengan kalimat

mencari 100 dokumen itu (masukkan 1 kalimat, ketemunya dokumen yg mana)

data tfidf ditransofrmasi menjadi sdikit dimensi menggunakan svd yg diambil v dan sigma (v tdk diambil smua)(Sebagian dri v) mentransformasi supaya dikit kolom

cosinus similtas (mirip mendekati 1) boleh pke euclidean

## Import Libraries

In [127]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive


## Mount Google Drive dan load data

In [128]:
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/ppw/tugas/DataTugas2/data_terbaru1.csv")
data = data[['Judul Berita', 'Isi Berita', 'Tanggal Berita', 'Kategori Berita']]
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,Pesan Anies ke Tom Lembong Tersangka Korupsi I...,Reporter\nNovali Panji Nugroho\nEditor\nEko Ar...,30-10-2024 05:55,Nasional
1,"Tom Lembong Tersangka Korupsi Impor Gula, Nama...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 23:55,Nasional
2,"Profil Tom Lembong, Eks Mendag dan Co-Captain ...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 22:55,Nasional
3,Tom Lembong Sempat Unggah Hal Ini Sehari Sebel...,Reporter\nTempo.co\nEditor\nAndry Triyanto Tji...,30-10-2024 02:55,Nasional
4,"Prabowo Ingin Tingkatkan Pembangunan di Papua,...",Reporter\nVedro Imanuel G\nEditor\nAgung Seday...,29-10-2024 23:55,Bisnis


## Drop Rows with Missing Values

- Menghapus baris yang tidak memiliki isi berita di kolom Isi Berita.
- Memastikan semua konten dalam kolom Isi Berita bertipe string.

In [129]:
data = data.dropna(subset=['Isi Berita'])
data['Isi Berita'] = data['Isi Berita'].astype(str)
print(f"Jumlah data setelah menghapus baris kosong: {len(data)}")


Jumlah data setelah menghapus baris kosong: 99


<ipython-input-129-06d1254a6c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Isi Berita'] = data['Isi Berita'].astype(str)


## TF-IDF Vectorization

 Mengonversi teks Isi Berita menjadi matriks TF-IDF, yaitu representasi numerik berdasarkan bobot term-frequency.

In [130]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Isi Berita'])
print("Dimensi TF-IDF Matrix:", tfidf_matrix.shape)

Dimensi TF-IDF Matrix: (99, 2163)


Output ini menunjukkan ukuran matriks setelah proses TF-IDF.

## Dimensionality Reduction with SVD

In [131]:
svd = TruncatedSVD(n_components=50)
reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)
print("Dimensi Reduced TF-IDF Matrix:", reduced_tfidf_matrix.shape)

Dimensi Reduced TF-IDF Matrix: (99, 50)


Fungsi: Mengurangi dimensi dari matriks TF-IDF untuk meningkatkan efisiensi dengan TruncatedSVD, mengurangi dimensi menjadi 50 fitur.

## Query Input and Vectorization

In [132]:
query = input("Masukkan kalimat untuk mencari dokumen terkait: ")
query_vector = tfidf_vectorizer.transform([query])
query_reduced = svd.transform(query_vector)

Masukkan kalimat untuk mencari dokumen terkait: kementrian prabowo


## Calculate Cosine Similarity

Menghitung kesamaan antara query dan dokumen menggunakan cosine similarity.

In [133]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Misalkan query_reduced dan reduced_tfidf_matrix sudah didefinisikan
cos_similarities = cosine_similarity(query_reduced, reduced_tfidf_matrix).flatten()

# Bulatkan nilai cosine similarity hingga 3 digit setelah koma
cos_similarities_rounded = np.round(cos_similarities, decimals=3)

# Tampilkan 5 nilai teratas
print("Cosine Similarity :")
cos_similarities_rounded


Cosine Similarity :


array([-0.   ,  0.144,  0.   ,  0.   ,  0.148,  0.145,  0.   ,  0.   ,
       -0.   ,  0.071,  0.   ,  0.   ,  0.075,  0.534, -0.   ,  0.   ,
       -0.   ,  0.   ,  0.422, -0.   ,  0.144,  0.   ,  0.   ,  0.148,
        0.145,  0.   ,  0.   , -0.   ,  0.071,  0.   ,  0.   ,  0.075,
        0.534, -0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.422, -0.   ,
        0.144,  0.   ,  0.   ,  0.148,  0.145,  0.   ,  0.   , -0.   ,
        0.071,  0.   ,  0.   ,  0.075,  0.534, -0.   ,  0.   , -0.   ,
        0.   , -0.   ,  0.422, -0.   ,  0.144,  0.   ,  0.   ,  0.148,
        0.145,  0.   ,  0.   , -0.   ,  0.071,  0.   ,  0.   ,  0.075,
        0.534, -0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.422, -0.   ,
        0.144,  0.   ,  0.   ,  0.148,  0.145,  0.   ,  0.   , -0.   ,
        0.534, -0.   ,  0.   , -0.   ,  0.422,  0.148,  0.145,  0.   ,
        0.534, -0.   ,  0.   ])

## Sorting and Selecting Top Results

Mengambil 10 dokumen teratas berdasarkan nilai cosine similarity.

In [137]:
top_100_indices = cos_similarities_rounded.argsort()[-100:][::-1]
top_100_docs = data.iloc[top_100_indices].copy()
top_100_docs['Cosine Similarity'] = cos_similarities_rounded[top_100_indices]

## Remove Duplicates and Display Results with Snippet

- Menghapus duplikat berdasarkan Judul Berita.
- Menyusun ulang tabel untuk menampilkan 100 kata pertama dari Isi Berita pada setiap hasil.
- Mengurutkan hasil berdasarkan Cosine Similarity dan menampilkan hasil akhir.

In [144]:
import pandas as pd

# Sample data (assuming top_100_docs is already defined and processed)
top_100_docs = top_100_docs.drop_duplicates(subset='Judul Berita')
results_table = top_100_docs[['Judul Berita', 'Tanggal Berita', 'Kategori Berita', 'Cosine Similarity']].copy()

# Add a snippet of the news content
results_table['Isi Berita'] = top_100_docs['Isi Berita'].apply(lambda x: ' '.join(x.split()[:100]))

# Sort the results by Cosine Similarity
results_table = results_table.sort_values(by='Cosine Similarity', ascending=False)
results_table.reset_index(drop=True, inplace=True)

In [147]:
results_table

,Judul Berita,Tanggal Berita,Kategori Berita,Cosine Similarity,Isi Berita
0,Bisik-bisik Prabowo kepada Fahri Hamzah Wakil ...,30-10-2024 03:56,Bisnis,0.534,Reporter Sukma Kanthi Nurani Editor S. Dian An...
1,Cerita Anggito Abimanyu soal Prabowo Panggil J...,29-10-2024 23:58,Bisnis,0.422,Reporter Nabiila Azzahra A Editor Rr. Ariyani ...
2,"Prabowo Ingin Tingkatkan Pembangunan di Papua,...",29-10-2024 23:55,Bisnis,0.148,Reporter Vedro Imanuel G Editor Agung Sedayu R...
3,Kementan Berencana Datangkan Sapi Hidup Ke Ind...,30-10-2024 00:56,Bisnis,0.145,Reporter M. Raihan Muzzaki Editor Aisha Shaidr...
4,"Tom Lembong Tersangka Korupsi Impor Gula, Nama...",29-10-2024 23:57,Nasional,0.144,Reporter Novali Panji Nugroho Editor Ahmad Fai...
5,Top Nasional: Presiden BEM FISIP Unair Berkoor...,30-10-2024 00:56,Nasional,0.075,"Reporter Tempo.co Editor Eko Ari Wibowo Rabu, ..."
6,Rekam Jejak Pernyataan Kontroversial Wamen Per...,30-10-2024 04:57,Nasional,0.071,Reporter Sukma Kanthi Nurani Editor S. Dian An...
7,Hasil Seleksi Administrasi PPPK 2024 Sudah Diu...,30-10-2024 05:56,Nasional,0.000,Reporter Hendrik Yaputra Editor Eko Ari Wibowo...
8,Profil Tom Lembong: Mendag Pemerintahan Jokowi...,29-10-2024 17:57,Bisnis,0.000,"Reporter Antara Editor Yudono Yanuar Selasa, 2..."
9,"Tom Lembong Tersangka Korupsi Impor Gula, Anie...",30-10-2024 06:57,Nasional,0.000,Reporter Novali Panji Nugroho Editor Devy Erni...
